###### Starting balance: $0.18
###### Ending balance: $1.28

### 0. Exploratory 

In [45]:
import pandas as pd 

df = pd.read_csv('data/input_data_revised.csv')
df.head()

,ad_cv_id,question,answer
0,154645787-441320,có con đang học hay đang làm?,Chưa già đình ạ [ 3 D ]
1,154645787-460036,có con đang học hay đang làm?,Chưa lập gia đình
2,154645787-489112,có con đang học hay đang làm?,Không
3,154645787-2858590,có con đang học hay đang làm?,Không có
4,154645787-2865609,có con đang học hay đang làm?,con đi học lop 5


### 1. Setup

In [3]:
!pip install langchain 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 60.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


In [9]:
!pip install langchain-community  # This is an example, the actual package name might differ

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.6 MB/s eta 0:00:00


In [4]:
!pip install openai

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


### 2. Prepare Prompt

In [32]:
template = '''
ROLE: You are a recruiting assistant tasked with evaluating the relevance of answers to job-related questions.

RELEVANT ANSWER: An answer is relevant if it directly addresses the question.

CONTEXT:
- Inputs usually include one Question-Answer pair, but there may be more.
- New lines in answers can indicate separate ideas but not always.
- Answers may include Vietnamese abbreviations (e.g., 'cf' for coffee, '23-25k' means VND 23000-25000/hour, '25k' means 25000 VND, 'k' alone means 'no').
- Answers can be in different forms of Vietnamese (with or without diacritics), and may include misspellings.
- Answers can be fully relevant, partially relevant, or irrelevant.
- Questions may reference locations in Vietnam (e.g., Binh Thanh is a district in Ho Chi Minh City) and private company or software names (e.g., 'Misa', 'Grab', 'Gojek').

EXAMPLES:

1. Relevant Answer -> Score: 1.00  
   Question: "bạn cam kết làm ít nhất 6 tháng được chứ? mức lương mong muốn của bạn?"  
   Answer: "Dạ cam kết ạ Mức lương e mong muốn là 6tr/1 tháng ạ"

2. Relevant Answer -> Score: 1.00  
   Question: "các loại nước uống nào là thế mạnh của bạn?"  
   Answer: "các loại juice, smoothie"

3. Partially Relevant Answer -> Score: 0.49  
   Question: "bạn cam kết làm ít nhất 6 tháng được chứ? mức lương mong muốn của bạn?"  
   Answer: "dạ"

4. Irrelevant Answer -> Score: 0.00  
   Question: "bạn có biết đánh máy pc thành thạo?"  
   Answer: "A\nV"

TASK: Provide a relevance score from 0.00 to 1.00 based on the provided context. 
- Return only the score (e.g., '0.25').
- Do not invent information or provide explanations.

INPUT:  
Question: {question}  
Answer: {answer}

'''


from langchain.chat_models import ChatOpenAI 
from langchain import PromptTemplate 

prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=template,
)

#### 3.1 Example of 1 Q&A pair

In [5]:
from langchain.chat_models import ChatOpenAI 


question = 'khi nào bạn có thể bắt đầu làm việc?'
answer = 'Có thể làm việc ngay khi phỏng vấn đạt'

prompt_formatted = prompt.format(question = question 
                                ,answer = answer)

# generate answer 
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)  # temperature is a hyperparameter that controls the randomness of the output, temperature=0 means deterministic output
answer = llm.predict(prompt_formatted) 
print(answer)

/tmp/ipykernel_28424/756228252.py:11: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = llm.predict(prompt_formatted)


Score: 1.0


#### 3.2 Apply on the whole df

In [50]:
df =df.head(200)
df.head()

,ad_cv_id,question,answer,score
0,154645787-441320,có con đang học hay đang làm?,Chưa già đình ạ [ 3 D ],None
1,154645787-460036,có con đang học hay đang làm?,Chưa lập gia đình,None
2,154645787-489112,có con đang học hay đang làm?,Không,None
3,154645787-2858590,có con đang học hay đang làm?,Không có,None
4,154645787-2865609,có con đang học hay đang làm?,con đi học lop 5,None


In [60]:
from langchain.chat_models import ChatOpenAI
from tqdm import tqdm
from functools import lru_cache

# Initialize the model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Static context (role and instructions) as system prompt
system_prompt = """
ROLE: You are a recruiting assistant tasked with evaluating the relevance of answers to job-related questions.

RELEVANT ANSWER: An answer is relevant if it directly addresses the question.

CONTEXT:
- Inputs usually include one Question-Answer pair, but there may be more.
- New lines in answers can indicate separate ideas but not always.
- Answers may include Vietnamese abbreviations (e.g., 'cf' for coffee, '23-25k' means VND 23000-25000/hour, '25k' means 25000 VND, 'k' alone means 'no').
- Answers can be in different forms of Vietnamese (with or without diacritics), and may include misspellings.
- Answers can be fully relevant, partially relevant, or irrelevant.
- Questions may reference locations in Vietnam (e.g., Binh Thanh is a district in Ho Chi Minh City) and private company or software names (e.g., 'Misa', 'Grab', 'Gojek').

EXAMPLES:

1. Relevant Answer -> Score: 1.00  
    INPUT:
    Question: "bạn cam kết làm ít nhất 6 tháng được chứ? mức lương mong muốn của bạn?"  
    Answer: "Dạ cam kết ạ Mức lương e mong muốn là 6tr/1 tháng ạ"
    OUTPUT: 1.00 

2. Relevant Answer -> Score: 1.00  
    INPUT:
    Question: "các loại nước uống nào là thế mạnh của bạn?"  
    Answer: "các loại juice, smoothie"
    OUTPUT: 1.00

3. Partially Relevant Answer -> Score: 0.49  
    INPUT:
    Question: "bạn cam kết làm ít nhất 6 tháng được chứ? mức lương mong muốn của bạn?"  
    Answer: "dạ"
    OUTPUT: 0.49

4. Irrelevant Answer -> Score: 0.00  
    INPUT:
    Question: "bạn có biết đánh máy pc thành thạo?"  
    Answer: "A\nV"
    OUTPUT: 0.00 

--- 

TASK: Provide a relevance score from 0.00 to 1.00 based on the provided context. 
- Return only the score (e.g., '0.25').
- Do not invent information or provide explanations.
"""


# Cache decorator to store results of previous computations
@lru_cache(maxsize=1024)  # Adjust maxsize as per your cache requirements
def get_cached_score(question, answer):
    try:
        # Construct the full prompt by concatenating the system context and the question-answer pair
        full_prompt = f"{system_prompt}\nINPUT:\nQuestion: {question}\nAnswer: {answer}"
        
        # Use the predict method to get the result
        response = llm.predict(full_prompt)
        
        # Return the response as the score (assuming it's a single line result)
        return response.strip()
    except Exception as e:
        print(f"Failed to get score for question: '{question}', error: {str(e)}")
        return None  # Handle errors gracefully

def get_scores(questions, answers):
    # Use caching and send the static context only once per batch
    scores = [get_cached_score(q, a) for q, a in zip(questions, answers)]
    return scores

# Assuming df is your DataFrame
batch_size = 100  # Define batch size suitable for your API rate limits and quotas
results = []

# Processing in batches with progress tracking
for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
    batch_questions = df['question'][i:i + batch_size].tolist()
    batch_answers = df['answer'][i:i + batch_size].tolist()
    batch_scores = get_scores(batch_questions, batch_answers)
    results.extend(batch_scores)

# Assign scores back to DataFrame
df['score'] = results


Processing batches:   0%|          | 0/2 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 2/2 [01:40<00:00, 50.29s/it]


In [62]:
df.tail()

,ad_cv_id,question,answer,score
195,158365068-2998513,bạn muốn làm việc trong môi trường như thế nào?,"hoà đồng , vui vẻ ạ",1.00
196,158365068-3003119,bạn đang tìm kiếm việc phù hợp để gắn bó lâu d...,Dạ em đang tìm kiếm một công việc phù hợp để g...,1.00
197,158365068-3003119,ưu điểm và nhược điểm của bạn?,"Ưu điểm: Học hỏi nhanh, chịu khó\nNhược điểm: ...",1.00
198,158365068-3003119,bạn muốn làm việc trong môi trường như thế nào?,Môi trường nào cũng có thể linh hoạt để hoà nhập,0.75
199,158365068-3003936,bạn đang tìm kiếm việc phù hợp để gắn bó lâu d...,Gắn bó lâu dài ạ,1.00


In [64]:
# RUN BY BATCH, FULL PROMPT 


from functools import lru_cache
import pandas as pd
from langchain.chat_models import ChatOpenAI

# Initialize the model
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Cache decorator to store results of previous computations
@lru_cache(maxsize=1024)  # Adjust maxsize as per your cache requirements
def get_cached_score(prompt):
    try:
        return llm.predict(prompt)
    except Exception as e:
        print("Failed to get score for prompt:", prompt, "Error:", str(e))
        return None  # Consider a default score or np.nan based on your requirements

def get_scores(questions, answers):
    prompts = [prompt.format(question=q, answer=a) for q, a in zip(questions, answers)]
    # Utilize caching function within list comprehension
    scores = [get_cached_score(prompt) for prompt in prompts]
    return scores

# Assuming df is your DataFrame
batch_size = 50  # Define batch size suitable for your API rate limits and quotas
results = []

# Processing in batches
for i in range(0, len(df), batch_size):
    batch_questions = df['question'][i:i + batch_size].tolist()
    batch_answers = df['answer'][i:i + batch_size].tolist()
    batch_scores = get_scores(batch_questions, batch_answers)
    results.extend(batch_scores)

# Assign scores back to DataFrame
df['score'] = results


In [65]:
export_time = pd.Timestamp.now().strftime('%Y%m%d%H%M%S') # Get current timestamp for unique filename
df.to_csv(f'data/output_data_{export_time}.csv', index=False, encoding='utf-8-sig')  # Save the DataFrame to CSV file